In [1]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

from RobotClass import Robot

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

In [33]:
import cv2
import numpy as np
import time
from drivers import PCA9685, Motor
from traitlets.config.configurable import SingletonConfigurable
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys

# Function to convert BGR image to JPEG for displaying in the widget
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def line_detection(image, lower_color, upper_color):

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

 

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        moment = cv2.moments(largest_contour)

        if moment["m00"] != 0:
            cx = int(moment["m10"] / moment["m00"])
            cy = int(moment["m01"] / moment["m00"])
            return (True, cx, cy, largest_contour)

    return (False, None, None, None)

 

def control_robot(robot, x, y, image_width, has_checked, t):
    
    if x is not None and y is not None:
        t = 0.1
        center = image_width // 2
        error = x - center

        if abs(error) < 90:
            robot.forward(0.6)
        elif error < -50:
            robot.forward(0.4)
            time.sleep(0.02)
            robot.forward_left(0.4)
            time.sleep(0.02)
            robot.left(0.5)
        elif error > 50:
            robot.forward(0.4)
            time.sleep(0.02)
            robot.forward_right(0.4)
            time.sleep(0.02)
            robot.right(0.5)
    elif t >= 0.3:
        robot.backward(0.2)
        t = 0.1
        time.sleep(0.2)
        has_checked = False
    elif not has_checked:
        robot.left(0.6)
        time.sleep(t)
        has_checked = True
        t += 0.05
    elif has_checked:
        robot.right(0.6)
        time.sleep(t)
        has_checked = False
        t += 0.05
        
    return has_checked, t

 

def main():
    
    # Create the image widget for displaying the frame with contour

    image_widget = widgets.Image(format='jpeg', width=300, height=300)

    display(image_widget)
    
    has_checked = False
    t = 0.1

    # Initialize the robot and camera

    robot = Robot()

    # Define the color range for the line (e.g., yellow or blue)

    lower_color = np.array([20, 100, 100])  # Lower bound of the yellow color in HSV
    upper_color = np.array([30, 255, 255])  # Upper bound of the yellow color in HSV
    
#     lower_color = np.array([110, 50, 50])
#     upper_color = np.array([160, 255, 255])

    try:
        while True:

            # Get the current frame from the camera
            frame = camera.value[175:,:,:]

            # Detect the line's position in the frame
            found, x, y, largest_contour = line_detection(frame, lower_color, upper_color)
            
            # Draw the largest contour on the frame
            if found:
                cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 3)

            # Update the image widget with the modified frame
            image_widget.value = bgr8_to_jpeg(frame)
            
            if found and has_checked:
                has_checked = not has_checked
            elif not found and has_checked:
                has_checked = not has_checked

            # Control the robot based on the line's position
            has_checked = control_robot(robot, x, y, frame.shape[1], has_checked, t)

            # Sleep for a while to reduce CPU usage
            time.sleep(0.1)

    except KeyboardInterrupt:
        print("Interrupted")

    finally:
        robot.stop()


if __name__ == "__main__":
    main()

Image(value=b'', format='jpeg', height='300', width='300')

Interrupted


In [28]:
robot = Robot()
robot.stop()

In [16]:

import cv2
import numpy as np
import time
from drivers import PCA9685, Motor
from traitlets.config.configurable import SingletonConfigurable
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys

# Function to convert BGR image to JPEG for displaying in the widget
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def line_detection(image, lower_color, upper_color):

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

 

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        moment = cv2.moments(largest_contour)

        if moment["m00"] != 0:
            cx = int(moment["m10"] / moment["m00"])
            cy = int(moment["m01"] / moment["m00"])
            return (True, cx, cy, largest_contour)

    return (False, None, None, None)

 

def control_robot(robot, x, y, image_width, has_checked, t):
    
#     last_turn = False is left, if True is right
    
    if x is not None and y is not None:
        t = 0.05
        center = image_width // 2
        error = x - center

        if abs(error) < 90:
            robot.forward(0.4)
        elif error < -50:
            robot.forward(0.4)
            time.sleep(0.02)
            robot.forward_left(0.4)
            time.sleep(0.02)
            robot.left(0.5)
            has_checked = True
        elif error > 50:
            robot.forward(0.4)
            time.sleep(0.02)
            robot.forward_right(0.4)
            time.sleep(0.02)
            robot.right(0.5)
            has_checked = False
    elif t >= 0.3:
        robot.backward(0.2)
        t = 0.01
        time.sleep(0.2)
        has_checked = not has_checked
        
    elif not has_checked:
        robot.left(0.7)
        time.sleep(t)
        has_checked = not has_checked
        t += 0.05
    elif has_checked:
        robot.right(0.7)
        time.sleep(t)
        has_checked = not has_checked
        t += 0.05
        
    return has_checked, t

 

def main():
    
    # Create the image widget for displaying the frame with contour

    image_widget = widgets.Image(format='jpeg', width=300, height=300)

    display(image_widget)
    
    has_checked = False
    last_turn = None
    t = 0.1

    # Initialize the robot and camera

    robot = Robot()

    # Define the color range for the line (e.g., yellow or blue)

    lower_color = np.array([20, 100, 100])  # Lower bound of the yellow color in HSV
    upper_color = np.array([30, 255, 255])  # Upper bound of the yellow color in HSV
    
#     lower_color = np.array([100, 50, 50])
#     upper_color = np.array([160, 255, 255])
    

    try:
        while True:

            # Get the current frame from the camera
            frame = camera.value[325:,:,:]
    
            # Detect the line's position in the frame
            found, x, y, largest_contour = line_detection(frame, lower_color, upper_color)
                    
            # Draw the largest contour on the frame
            if found:
                cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 3)

            # Update the image widget with the modified frame
            image_widget.value = bgr8_to_jpeg(frame)
            
            if found and has_checked:
                has_checked = not has_checked
            elif not found and has_checked:
                has_checked = not has_checked

            # Control the robot based on the line's position
            has_checked = control_robot(robot, x, y, frame.shape[1], has_checked, t)

            # Sleep for a while to reduce CPU usage
            time.sleep(0.1)

    except KeyboardInterrupt:
        print("Interrupted")

    finally:
        robot.stop()


if __name__ == "__main__":
    main()

Image(value=b'', format='jpeg', height='300', width='300')

Interrupted


In [ ]:
import cv2
import numpy as np
import time
from drivers import PCA9685, Motor
from traitlets.config.configurable import SingletonConfigurable
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys

# Function to convert BGR image to JPEG for displaying in the widget
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# This function detects the line in the image using color thresholding and contour detection
def line_detection(image, lower_color, upper_color):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Create a binary mask where color of the line falls within the bounds
    mask = cv2.inRange(hsv, lower_color, upper_color)
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If any contour is found
    if len(contours) > 0:
        # Find the largest contour based on area
        largest_contour = max(contours, key=cv2.contourArea)
        # Calculate moments of the largest contour
        moment = cv2.moments(largest_contour)

        # If the contour has area (prevents divide by zero error)
        if moment["m00"] != 0:
            # Calculate centroid of the contour
            cx = int(moment["m10"] / moment["m00"])
            cy = int(moment["m01"] / moment["m00"])
            # Return flag, centroid and the contour itself
            return (True, cx, cy, largest_contour)

    # If no contour found, return False and None values
    return (False, None, None, None)

# This function controls the robot based on the position of the line
def control_robot(robot, x, y, image_width, has_checked, t):
    # If line is detected
    if x is not None and y is not None:
        # Calculate error between line's position and center of the frame
        center = image_width // 2
        error = x - center

        # If line is close to center, move forward
        if abs(error) < 90:
            robot.forward(0.6)
        # If line is to the left, turn left
        elif error < -50:
            robot.forward(0.4)
            time.sleep(0.02)
            robot.forward_left(0.4)
            time.sleep(0.02)
            robot.left(0.5)
        # If line is to the right, turn right
        elif error > 50:
            robot.forward(0.4)
            time.sleep(0.02)
            robot.forward_right(0.4)
            time.sleep(0.02)
            robot.right(0.5)
    # If no line detected
    else:
        # Do a left-right scanning motion to try and find the line
        if not has_checked:
            robot.left(0.6)
            time.sleep(t)
            has_checked = True
            t += 0.05
        elif has_checked:
            robot.right(0.6)
            time.sleep(t)
            has_checked = False
            t += 0.05
        
    return has_checked, t

# The main function where robot and camera are initialized and control loop is run
def main():

    # Create an image widget for displaying frame with contour
    image_widget = widgets.Image(format='jpeg', width=300, height=300)

    # Display the image widget in the Jupyter notebook
    display(image_widget)
    
    # Initialize boolean for checking line detection and a time variable
    has_checked = False
    t = 0.1

    # Initialize the robot and camera
    robot = Robot()

    # Define the color range for the line (Yellow)
    lower_color1 = np.array([20, 100, 100])  # Lower bound of the yellow color in HSV
    upper_color1 = np.array([30, 255, 255])  # Upper bound of the yellow color in HSV

    # Define the color range for the line (Blue)
    lower_color2 = np.array([110, 50, 50])  # Lower bound of the yellow color in HSV
    upper_color2 = np.array([160, 255, 255])  # Upper bound of the yellow color in HSV

    # Begin main control loop
    try:
        while True:
            # Get the current frame from the camera
            frame = camera.value[175:,:,:]

            # Detect the yellow line's position in the frame 
            found1, x1, y1, largest_contour1 = line_detection(frame, lower_color1, upper_color1)

            # Detect the Bue line's position in the frame 
            found2, x2, y2, largest_contour2 = line_detection(frame, lower_color2, upper_color2)
            
            # Checks which color has the largest contour
            if max(largest_contour1,largest_contour2) == largest_contour1:
                largest_contour, x, y = largest_contour1, x1, y1
            elif max(largest_contour1,largest_contour2) == largest_contour2:
                largest_contour, x, y = largest_contour2, x2, y2

            # If a line is detected, draw the largest contour on the frame
            if found1 and found2:
                cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 3)

            # Update the image widget with the modified frame
            image_widget.value = bgr8_to_jpeg(frame)
            
            # If line was found and has_checked is true, set has_checked to false, and vice versa
            if found and has_checked:
                has_checked = not has_checked
            elif not found and has_checked:
                has_checked = not has_checked

            # Control the robot based on the line's position
            has_checked, t = control_robot(robot, x, y, frame.shape[1], has_checked, t)

            # Sleep for a while to reduce CPU usage
            time.sleep(0.1)

    # If the control loop is interrupted, stop the robot
    except KeyboardInterrupt:
        print("Interrupted")
        robot.stop()

    # If an unhandled exception occurs, also stop the robot
    finally:
        robot.stop()

# Run the main function if this script is run directly
if __name__ == "__main__":
    main()
